In [1]:
#정규식, 날짜형식 모듈
import re
import datetime
#플라스크 프레임워크, 플라스크 로그인 모듈
from flask import *
from flask_login import *
#langchain-huggingface word2vector 모듈
from langchain_huggingface import HuggingFaceEmbeddings
#oracleDB api, 사용자, 게시글 정보 저장
import oracledb
#postgreSQL api, 임베딩한 벡터 저장
import psycopg2
from psycopg2 import pool

In [2]:
#리스트, 튜플 형식의 사용자 데이터를 딕셔너리로 매핑
def userdata2dict(d, j=[], l=[], f=[], fe=[]):
    return {
        'ID': d[0],
        'PW': '-',
        'NAME': d[2],
        'NICK': d[3],
        'BIRTH': d[4].strftime('%Y-%m-%d'),
        'IMG_ID': d[5],
        'JOINED_AT': d[6].strftime('%Y-%m-%d %H:%M:%S'),
        'TYPE': d[7],
        'JOIN_CH': j,
        'LIKE_CH': l,
        'FOLLOW': f,
        'FOLLOWER': fe
    }
#리스트, 튜플 형식의 챌린지 데이터를 딕셔너리로 매핑
def chaldata2dict(d, j=[], l=[], p=[], auto=False):
    if auto:
        chal_idx = d[0]
        with con.cursor() as cur:
            data = cur.execute(
                "SELECT MB_EMAIL FROM JOIN_CHALLENGES WHERE CHAL_IDX=:1",
                (chal_idx,)
            ).fetchall()
            j = [d[0] for d in data] if data != None else []
            data = cur.execute(
                "SELECT MB_EMAIL FROM LIKE_CHALLENGES WHERE CHAL_IDX=:1",
                (chal_idx,)
            ).fetchall()
            l = [d[0] for d in data] if data != None else []
            data = cur.execute(
                "SELECT POST_IDX FROM POSTS WHERE CHAL_IDX=:1 ORDER BY POSTED_AT",
                (chal_idx,)
            ).fetchall()
            p = [d[0] for d in data] if data != None else []
    return {
        'ID': d[0],
        'TITLE': d[1],
        'DETAIL': d[2].read(),
        'CREATED_AT': d[3].strftime('%Y-%m-%d %H:%M:%S'),
        'ADMIN': d[4],
        'IMG_ID': d[5],
        'JOIN_LIST': j,
        'LIKE_LIST': l,
        'POST_LIST': p
    }
#리스트, 튜플 형식의 포스트 데이터를 딕셔너리로 매핑
def postdata2dict(d, l=[], c=[], auto=False):
    if auto:
        post_idx = d[0]
        with con.cursor() as cur:
            data = cur.execute(
                "SELECT MB_EMAIL FROM LIKE_POSTS WHERE POST_IDX=:1",
                (post_idx,)
            ).fetchall()
            l = ([d[0] for d in data] if data != None else [])
            data = cur.execute(
                "SELECT CMT_IDX FROM COMMENTS WHERE POST_IDX=:1 ORDER BY COMMENTED_AT",
                (post_idx,)
            ).fetchall()
            c = ([d[0] for d in data] if data != None else [])
    return {
        'ID': d[0],
        'TITLE': d[1],
        'CONTENT': d[2].read(),
        'POSTED_AT': d[3].strftime('%Y-%m-%d %H:%M:%S'),
        'VIEWS': d[4],
        'WRITER': d[5],
        'CHAL_IDX': d[6],
        'IMG_ID': d[7],
        'LIKE_LIST': l,
        'CMT_LIST': c
    }
#리스트, 튜플 형식의 댓글 데이터를 딕셔너리로 매핑
def cmtdata2dict(d, l=[]):
    return {
        'ID': d[0],
        'POST_IDX': d[1],
        'CONTENT': d[2],
        'COMMENTED_AT': d[3].strftime('%Y-%m-%d %H:%M:%S'),
        'WRITER': d[4],
        'LIKE_LIST': l
    }

In [3]:
#region MODEL, 모델 클래스 영역

#사용자 클래스
class User(UserMixin):
    def __init__(self, info):
        self.info = info

    #region getter
    def get_id(self):
        return self.info.get('ID')
    def get_name(self):
        return self.info.get('NAME')
    def get_nick(self):
        return self.info.get('NICK')
    def get_birth(self):
        return self.info.get('BIRTH')
    def get_img_id(self):
        return self.info.get('IMG_ID')
    def get_joinCh(self):
        return self.info.get('JOIN_CH')
    def get_likeCh(self):
        return self.info.get('LIKE_CH')
    def get_follow(self):
        return self.info.get('FOLLOW')
    def get_follower(self):
        return self.info.get('FOLLOWER')
    #endregion
    
    @staticmethod
    def get_user_info(userEmail):
        with con.cursor() as cur:
            userData = cur.execute("select * from members where mb_email=:1", (userEmail,)).fetchone()
            if userData == None:
                return None
            userData = list(userData)
            data = cur.execute("SELECT CHAL_IDX FROM JOIN_CHALLENGES WHERE MB_EMAIL=:1", (userEmail,)).fetchall()
            j = [d[0] for d in data] if data != None else []
            data = cur.execute("SELECT CHAL_IDX FROM LIKE_CHALLENGES WHERE MB_EMAIL=:1", (userEmail,)).fetchall()
            l = [d[0] for d in data] if data != None else []
            data = cur.execute(
                "SELECT FOLLOW FROM FOLLOWS WHERE FOLLOWER=:1",
                (userEmail,)
            )
            f = [d[0] for d in data] if data != None else []
            data = cur.execute(
                "SELECT FOLLOWER FROM FOLLOWS WHERE FOLLOW=:1",
                (userEmail,)
            )
            fe = [d[0] for d in data] if data != None else []
        return userdata2dict(userData, j=j, l=l, f=f, fe=fe)

    @staticmethod
    def get_user_info_simple(userEmail):
        with con.cursor() as cur:
            userData = cur.execute("select * from members where mb_email=:1", (userEmail,)).fetchone()
            if userData == None:
                return None
        return userdata2dict(userData)

#챌린지 클래스
class Challenge():
    def __init__(self, info):
        self.info = info
        
    #region getter
    def get_id(self):
        return self.info.get('ID')
    def get_title(self):
        return self.info.get('TITLE')
    def get_detail(self):
        return self.info.get('DETAIL')
    def get_created_at(self):
        return self.info.get('CREATED_AT')
    def get_admin(self):
        return self.info.get('ADMIN')
    def get_img(self):
        return self.info.get('IMG_ID')
    def get_join_list(self):
        return self.info.get('JOIN_LIST')
    def get_like_list(self):
        return self.info.get('LIKE_LIST')
    def get_post_list(self):
        return self.info_get('POST_LIST')
    #endregion
    
    @staticmethod
    def get_challenge_info(chal_idx):
        with con.cursor() as cur:
            chalData = cur.execute("SELECT * FROM CHALLENGES WHERE CHAL_IDX=:1", (chal_idx,)).fetchone()
            if chalData == None:
                return None
            chalData = list(chalData)
            data = cur.execute("SELECT MB_EMAIL FROM JOIN_CHALLENGES WHERE CHAL_IDX=:1", (chal_idx,)).fetchall()
            j = [d[0] for d in data] if data != None else []
            data = cur.execute("SELECT MB_EMAIL FROM LIKE_CHALLENGES WHERE CHAL_IDX=:1", (chal_idx,)).fetchall()
            l = [d[0] for d in data] if data != None else []
            data = cur.execute("SELECT POST_IDX FROM POSTS WHERE CHAL_IDX=:1 ORDER BY POSTED_AT", (chal_idx,)).fetchall()
            p = [d[0] for d in data] if data != None else []
        return chaldata2dict(chalData, j, l, p)

    @staticmethod
    def get_challenge_info_simple(chal_idx):
        with con.cursor() as cur:
            d = cur.execute("SELECT * FROM CHALLENGES WHERE CHAL_IDX=:1", (chal_idx,)).fetchone()
            if d == None:
                return None
        return chaldata2dict(d)

#포스트 클래스
class Post():
    def __init__(self, info):
        self.info = info
    
    #region getter
    def get_id(self):
        return self.info.get('ID')
    def get_title(self):
        return self.info.get('TITLE')
    def get_content(self):
        return self.info.get('DETAIL')
    def get_posted_at(self):
        return self.info.get('POSTED_AT')
    def get_views(self):
        return self.info.get('VIEWS')
    def get_writer(self):
        return self.info.get('WRITER')
    def get_chal_idx(self):
        return self.info.get('CHAL_IDX')
    def get_img_id(self):
        return self.info.get('IMG_ID')
    def get_like_list(self):
        return self.info.get('LIKE_LIST')
    def get_cmt_list(self):
        return self.info.get('CMT_LIST')
    #endregion

    @staticmethod
    def get_post_info(post_idx):
        with con.cursor() as cur:
            postData = cur.execute("SELECT * FROM POSTS WHERE POST_IDX=:1",(post_idx,)).fetchone()
            if postData == None:
                return None
            postData = list(postData)
            data = cur.execute("SELECT MB_EMAIL FROM LIKE_POSTS WHERE POST_IDX=:1", (post_idx,)).fetchall()
            l = ([d[0] for d in data] if data != None else [])
            data = cur.execute(
                "SELECT CMT_IDX FROM COMMENTS WHERE POST_IDX=:1 ORDER BY COMMENTED_AT",
                (post_idx,)
            ).fetchall()
            c = ([d[0] for d in data] if data != None else [])
        return postdata2dict(postData, l, c)

    @staticmethod
    def get_post_info_simple(post_idx):
        with con.cursor() as cur:
            d = cur.execute("SELECT * FROM POSTS WHERE POST_IDX=:1",(post_idx,)).fetchone()
            if d == None:
                return None
            like = cur.execute("SELECT count(MB_EMAIL) FROM LIKE_POSTS WHERE POST_IDX=:1", (post_idx,)).fetchone()[0]
        return {
            'ID': post_idx,
            'TITLE': d[1],
            'VIEWS': d[4],
            'WRITER': d[5],
            'LIKE': like
        }

#댓글 클래스
class Comment():
    def __init__(self, info):
        self.info = INFO
    
    #region getter

    #endregion

    @staticmethod
    def get_comment_info(cmt_idx):
        with con.cursor() as cur:
            cmtData = cur.execute("SELECT * FROM COMMENTS WHERE CMT_IDX=:1", (cmt_idx,)).fetchone()
            if cmtData == None:
                return None
            cmtData = list(cmtData)
            data = cur.execute("SELECT MB_EMAIL FROM LIKE_COMMENTS WHERE CMT_IDX=:1", (cmt_idx,)).fetchall()
            l = [d[0] for d in data] if data != None else []
        return cmtdata2dict(cmtData, l)
    
#endregion

#region INIT, 초기화 영역

#플라스크 세션 암호화 설정
app = Flask(__name__)
app.secret_key = 'qalgmwjnfcndlemfyporpsltems'

#플라스크 로그인 모듈 기본 설정
lm = LoginManager()
lm.init_app(app)

@lm.user_loader
def user_loader(userId):
    userInfo = User.get_user_info(userId)
    return User(userInfo)

@lm.unauthorized_handler
def unauthorized():
    return redirect('/')

#허깅페이스 모델 초기화
HFmodel = HuggingFaceEmbeddings(
    model_name='jhgan/ko-sroberta-nli',
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True},
)

#호스팅할 주소, 이미지 서버 주소
host = 'localhost'
imgserver = 'localhost:5051'

#endregion

#region VIEW, 뷰페이지 영역

#로그인 페이지
@app.route('/loginform')
def loginform():
    if current_user.is_authenticated:
        #로그인 된 상태인 경우 홈으로 리디렉션
        return redirect('/')
    else:
        #로그인 안된 경우 로그인 페이지로 이동
        return render_template('loginform.html')

#회원가입 페이지
@app.route('/joinform')
def joinform():
    if current_user.is_authenticated:
        #로그인 된 상태인 경우 홈으로 리디렉션
        return redirect('/')
    else:
        #로그인 안된 경우 회원가입 페이지로 이동
        return render_template(
            'joinform.html',
            imgserver = imgserver
        )

#챌린지 상세 페이지
@app.route('/challenge', methods = ['get', 'post'])
def challenge():
    index = request.args.get('index', 1)
    chalData = Challenge.get_challenge_info(index)
    #존재하지 않는 챌린지인 경우 홈으로 이동
    if chalData == None:
        return redirect('/')
    adminName = User.get_user_info_simple(chalData['ADMIN'])['NICK']
    #챌린지 개설자, 정보, 이미지서버 전달
    return render_template(
        'challenge.html',
        adminName = adminName,
        chInfo = chalData,
        imgserver = imgserver
    )

#DEPRECATED
# @app.route('/challenge/postform', methods = ['get', 'post'])
# @login_required
# def postform():
#     index = request.args.get('index', 1)
#     chInfo = Challenge.get_challenge_info(index)
#     if chInfo == None:
#         return redirect('/')
#     return render_template(
#         'newpostform.html',
#         chInfo = chInfo,
#         imgserver = imgserver
#     )

#포스트 상세 페이지
@app.route('/post', methods = ['get', 'post'])
def post():
    index = request.args.get('index', 1)
    postData = Post.get_post_info(index)
    #존재하지 않는 포스트인 경우 홈으로 이동
    if postData == None:
        return redirect('/')
    userInfo = {postData['WRITER']: User.get_user_info_simple(postData['WRITER'])}
    #포스트 작성자, 정보, 이미지 서버 전달
    return render_template(
        'post.html',
        postInfo = postData,
        userInfo = userInfo,
        imgserver = imgserver
    )

#<userId>의 상세 페이지
@app.route('/user/<userId>', methods = ['get', 'post'])
def userinfo(userId):
    userData = User.get_user_info(userId)
    menu = request.args.get('menu', '0')
    #존재하지 않는 유저인 경우 홈으로 이동
    if userData == None:
        return redirect('/')
    with con.cursor() as cur:
        #유저가 완료한 챌린지 정보 받아옴
        endList = cur.execute(
            "SELECT CHAL_IDX, END_IDX, JOINED_AT, ENDED_AT FROM END_CHALLENGES WHERE MB_EMAIL=:1",
            (userId,)
        ).fetchall()
    endList = [list(e) for e in endList]
    chalList = []
    #유저가 완료한, 좋아요, 참여중인 챌린지 목록
    chalList += userData['JOIN_CH']
    chalList += userData['LIKE_CH']
    chalList += [e[0] for e in endList]
    #중복되는 챌린지 제거
    chalList = set(chalList)
    if len(chalList) != 0:
        with con.cursor() as cur:
            data = cur.execute(
                f"select * from challenges where chal_idx in ({','.join([str(c) for c in chalList])})"
            ).fetchall()
        chalInfo = {d[0]: chaldata2dict(d, auto=True) for d in data}
    else:
        chalInfo = {}

    return render_template(
        'userpage.html',
        userInfo = userData, #유저의 정보
        chalInfo = chalInfo, #유저페이지에 보일 챌린지 정보
        endList = endList, #종료한 챌린지 목록
        imgserver = imgserver, #이미지 서버
        menu = menu #어느 메뉴에서 시작할지 전달
    )
    
#<userId>의 팔로우/팔로워 목록
@app.route('/follows/<userId>')
def userfollows(userId):
    userData = User.get_user_info(userId)
    #유저가 팔로우한 유저 + 유저를 팔로우한 유저 목록 중복제거
    followList = set(userData['FOLLOW'] +userData['FOLLOWER'])
    followInfo = {f: User.get_user_info_simple(f) for f in followList}
    return render_template(
        'follow.html',
        userInfo = userData, #유저 정보
        followInfo = followInfo, #유저가 팔로우/유저를 팔로우한 유저 정보
        imgserver = imgserver #이미지 서버
    )

#DEPRECATED
# @app.route('/userhome')
# @login_required
# def userhome():
#     userId = current_user.get_id()
#     userData = User.get_user_info(userId)
#     if userData == None:
#         return "ERROR"
#     with con.cursor() as cur:
#         data = cur.execute(
#             "SELECT * FROM POSTS WHERE MB_EMAIL IN (SELECT FOLLOW FROM FOLLOWS WHERE FOLLOWER=:1) ORDER BY POSTED_AT",
#             (userData['ID'],)
#         ).fetchall()
#         postInfo = {d[0]: postdata2dict(d) for d in data}
#     return render_template(
#         'userhome.html',
#         postInfo = postInfo,
#         imgserver = imgserver
#     )

#새로운 챌린지 작성 화면
@app.route('/newchallengeform') 
@login_required
def newchallengeform():
    return render_template(
        'newchallengeform.html',
        imgserver = imgserver
    )

#메인페이지
@app.route('/')
def index():
    chalList = []
    postInfo = {}
    if current_user.is_authenticated:
        #로그인한 유저의 경우
        userId = current_user.get_id()
        #참여중인 챌린지(4개), 코사인 유사도 기준으로 챌린지별 추천 점수 부여
        chalList += current_user.get_joinCh()[:4]
        pcon = dbcp.getconn()
        with pcon.cursor() as pcur:
            pcur.execute(
                f"select id, embedding<=>(select embedding from uservector where id='{userId}') from challengeembeddings"
            )
            res = pcur.fetchall()
        dbcp.putconn(pcon)
        score = {r[0]: 1-r[1] for r in res}
    else:
        #모든 챌린지에 1점 부여
        with con.cursor() as cur:
            res = cur.execute(
                "select chal_idx from challenges"
            ).fetchall()
        score = {r[0]: 1 for r in res}

    #챌린지별 좋아요, 참여자 수
    with con.cursor() as cur:
        likeRes = cur.execute("select chal_idx, count(chal_idx) from like_challenges group by chal_idx").fetchall()
        joinRes = cur.execute("select chal_idx, count(chal_idx) from join_challenges group by chal_idx").fetchall()
    likeRes = {r[0]: r[1] for r in likeRes}
    joinRes = {r[0]: r[1] for r in joinRes}
    
    #좋아요 하나당 0.1, 참여자 하나당 1로 챌린지 기본 점수에 곱함
    for i in score:
        score[i] *= (likeRes.get(i, 0)*0.1 +joinRes.get(i, 0))
    if current_user.is_authenticated:
        #유저가 참여중인 챌린지는 0점 부여(이미 참여중인 챌린지 추천X)
        for i in current_user.get_joinCh():
            score[i] = 0
    score = sorted(score, key=lambda x: -score[x])
    chalList += score[:4]

    chalList = set(chalList)
    return render_template(
        'mainpage.html',
        recList = score[:4], #추천리스트 앞에서 4개
        chalInfo = {i: Challenge.get_challenge_info(i) for i in chalList}, #페이지에 나타날 챌린지 정보
        postInfo = postInfo, #페이지에 나타날 포스트 정보
        imgserver = imgserver
    )

#검색 페이지
@app.route('/search')
def search():
    q = request.args.get('q', '')
    return render_template(
        'search.html',
        imgserver = imgserver,
        placeholder = q #검색어 전달
    )

# @app.route('/test')
# def test():
#     return render_template('testPage.html')
    
#endregion

#region CONTROLLER 동작 영역

#로그인 동작
@app.route('/login', methods = ['get', 'post'])
def login():
    params = request.get_json()
    userId = params['userId']
    userPw = params['userPw']
    #유효성 검사
    if  userId == '' or userPw == '':
        return jsonify({"result": 0, "msg": "아이디와 비밀번호를 입력해주세요."})
    
    with con.cursor() as cur:
        pwCheck = cur.execute(
            "select mb_email from members where mb_email=:1 and mb_pw=:2",
            (userId, userPw)
        ).fetchone()
    if pwCheck is None:
        return jsonify({"result": 0, "msg": "아이디 또는 비밀번호가 일치하지 않습니다."})
    
    userData = User.get_user_info(userId)
    loginUser = User(userData)
    login_user(loginUser)
    return jsonify({"result": 1, "msg": url_for('index')})
    #유효성검사 성공시 메인으로 이동

#로그아웃 동작
@app.route('/logout')
def logout():
    logout_user()
    return redirect('/')

#이메일 중복체크 동작
@app.route('/joinEmail', methods = ['get', 'post'])
def joinEmail():
    params = request.get_json()
    if params['id'] == '' or params['domain'] == '':
        return jsonify({"result": 0, "msg": "이메일을 입력해주세요."})
    
    userData = User.get_user_info(params['id'] +'@' +params['domain'])
    if userData is not None:
        return jsonify({"result": 0, "msg": "이미 사용 중인 이메일입니다."})
    else:
        return jsonify({"result": 1, "msg": "사용 가능한 이메일입니다."})

#회원 가입 동작
@app.route('/join', methods = ['get', 'post'])
def join():
    params = request.get_json()
    userId = params['ID']
    userPw = params['PW']
    userName = params['NAME']
    userNick = params['NICK']
    userBirth = params['BIRTH']
    userImgId = params['IMG_ID']

    emailRegulation = re.compile('^[\w\-\.]+\@[\w\-]+\.[\w\-]{2,4}$')
    pwRegulation = re.compile('^(?=.*[0-9])(?=.*[a-zA-Z])(?=.*[\!\@\#\$\^\&\*\(\)])[0-9a-zA-Z\!\@\#\$\^\&\*\(\)]{8,16}$')

    msg = {
        'result': 1,
        'email': 0,
        'pw': 0,
        'name': 0,
        'nick': 0,
        'birth': 0
    }
    #유효성 검사
    if User.get_user_info(userId):
        msg['result'] = 0
        msg['email'] = '이미 사용 중인 이메일입니다.'
    if emailRegulation.match(userId) is None:
        msg['result'] = 0
        msg['email'] = '유효하지 않은 이메일입니다.'

    if pwRegulation.match(userPw) is None:
        msg['result'] = 0
        msg['pw'] = '유효하지 않은 비밀번호입니다.'

    if userName == '':
        msg['result'] = 0
        msg['name'] = '이름은 비워둘 수 없습니다.'

    if userNick == '':
        msg['result'] = 0
        msg['nick'] = '이름은 비워둘 수 없습니다.'

    if userBirth == '--':
        msg['result'] = 0
        msg['birth'] = '생일은 비워둘 수 없습니다.'

    if msg['result'] == 0:
        return jsonify(msg)

    #유효성 검사 통과시 유저 데이터 저장
    with con.cursor() as cur:
        cur.execute("INSERT INTO MEMBERS VALUES(:1, :2, :3, :4, TO_DATE(:5, 'YYYY-MM-DD'), :6, SYSDATE, 'USER')", (userId, userPw, userName, userNick, userBirth, userImgId))
        con.commit()
    #초기 유저 벡터 생성
    pcon = dbcp.getconn()
    with pcon.cursor() as pcur:
        pcur.execute(
            f"INSERT INTO USERVECTOR VALUES('{userId}', '{[0 for _ in range(768)]}'::vector)"
        )
    pcon.commit()
    dbcp.putconn(pcon)
    #가입한 계정으로 로그인
    login_user(User(User.get_user_info(userId)))
    return jsonify({'result': 1, 'msg':'가입 성공'})

#챌린지 정보 반환
@app.route('/challengeinfo', methods = ['get', 'post'])
def challengeinfo():
    index = request.args.get('index', 1)
    return jsonify(Challenge.get_challenge_info(index))
#챌린지 좋아요 수 반환
@app.route('/challengeinfo/likeNum', methods = ['get', 'post'])
def challengeinfo_likeNum():
    index = request.args.get('index', 1)
    with con.cursor() as cur:
        data = cur.execute("SELECT MB_EMAIL FROM LIKE_CHALLENGES WHERE CHAL_IDX=:1", (index,)).fetchall()
    return jsonify({'likeNum': len(data)})
#챌린지 참여자 수 반환
@app.route('/challengeinfo/joinList', methods = ['get', 'post'])
def challengeinfo_joinList():
    index = request.args.get('index', 1)
    with con.cursor() as cur:
        data = cur.execute("SELECT MB_EMAIL FROM JOIN_CHALLENGES WHERE CHAL_IDX=:1", (index,)).fetchall()
    return jsonify({'joinList': [d[0] for d in data]})

#챌린지 개설 동작
@app.route('/newchallenge', methods = ['get', 'post'])
@login_required
def newchallenge():
    params = request.get_json()
    title = params['TITLE']
    detail = params['DETAIL']
    img_id = params['IMG_ID']
    if title == '' or detail == '':
        return jsonify({"result": 0})
    #챌린지 제목, 내용 word2vec 임베딩
    embeddingTitle = HFmodel.embed_query(title)
    embeddingDetail = HFmodel.embed_query(detail)
    #챌린지 정보 저장, 개설자 챌린지에 참여
    with con.cursor() as cur:
        cur.execute(
            "INSERT INTO CHALLENGES VALUES(0, :1, :2, SYSDATE, :3, :4)",
            (title, detail, current_user.get_id(), img_id)
        )
        idx = cur.execute("select max(chal_idx) from challenges").fetchone()[0]
        cur.execute(
            "INSERT INTO JOIN_CHALLENGES VALUES(:1, :2, SYSDATE)",
            (current_user.get_id(), idx)
        )
        chal_idx = cur.execute(
                "select max(chal_idx) from challenges"
            ).fetchone()[0]
        con.commit()
    #제목, 내용의 벡터 합으로 챌린지 초기 벡터 생성
    pcon = dbcp.getconn()
    with pcon.cursor() as pcur:
        pcur.execute(
            f"insert into challengeembeddings values ({chal_idx}, '{embeddingTitle}'::vector)"
        )
        pcur.execute(
            f"update challengeembeddings set embedding = challengeembeddings.embedding+'{embeddingDetail}'::vector where id={idx}"
        )
    pcon.commit()
    dbcp.putconn(pcon)
    return jsonify({"result": 1})

#챌린지 좋아요, 참여 동작
@app.route('/checkch', methods = ['get', 'post'])
@login_required
def checkch():
    index = int(request.args.get('index', 1))
    typ = request.args.get('type', '')
    userId = current_user.get_id()
    with con.cursor() as cur:
        if typ == '1': #좋아요 동작
            if index in current_user.get_likeCh(): #좋아요를 이미 누른 상태면
                #좋아요 취소
                cur.execute("DELETE FROM LIKE_CHALLENGES WHERE MB_EMAIL=:1 AND CHAL_IDX=:2", (userId, index))
            else:
                #좋아요
                cur.execute("INSERT INTO LIKE_CHALLENGES VALUES(:1, :2)", (userId, index))
            likenum = cur.execute(
                "select count(chal_idx) from like_challenges where chal_idx=:1",
                (index,)
            ).fetchone()[0]
            con.commit()
            return jsonify({'result': 1, "likenum": likenum}) #좋아요 수 반환
        elif typ == '2': #참여 동작
            if index in current_user.get_joinCh(): #이미 참여중이면
                joinedAt = cur.execute(
                    "SELECT JOINED_AT FROM JOIN_CHALLENGES WHERE MB_EMAIL=:1 AND CHAL_IDX=:2",
                    (userId, index)
                ).fetchone()[0]
                cur.execute(
                    "DELETE FROM JOIN_CHALLENGES WHERE MB_EMAIL=:1 AND CHAL_IDX=:2",
                    (userId, index)
                ) #참여 해제
                if joinedAt.strftime('%Y-%m-%d') != datetime.datetime.now().strftime('%Y-%m-%d'): #당일 참여, 당일 취소가 아니면
                    cur.execute(
                        "INSERT INTO END_CHALLENGES VALUES(:1, :2, :3, SYSDATE, 0)",
                        (userId, index, joinedAt)
                    )
                con.commit() #참여 기록 저장
                return jsonify({'result': 1})
            else: #챌린지 참여
                cur.execute("INSERT INTO JOIN_CHALLENGES VALUES(:1, :2, sysdate)", (userId, index))
                con.commit()
                return jsonify({'result': 1})
    return jsonify({'result': 0})

#유저 팔로우 동작
@app.route('/follow', methods = ['get', 'post'])
@login_required
def follow():
    userId = request.args.get('id', '')
    nowId = current_user.get_id()
    if userId == '':
        return jsonify({'result': 0})
    follower = User.get_user_info(userId)['FOLLOWER']

    with con.cursor() as cur:
        if nowId in follower: #이미 팔로우 중인 대상이면
            cur.execute(
                "DELETE FROM FOLLOWS WHERE FOLLOW=:1 AND FOLLOWER=:2",
                (userId, nowId)
            ) #팔로우 해제
            isFollow = 0
        else:
            cur.execute(
                "INSERT INTO FOLLOWS VALUES(:1, :2)",
                (userId, nowId)
            ) #팔로우
            isFollow = 1
        con.commit()
    return jsonify({'result': 1, 'isFollow': isFollow}) #팔로우 여부 반환

#팔로우 중인지 확인
@app.route('/followcheck', methods = ['get', 'post'])
@login_required
def followcheck():
    userId = request.args.get('id', '')
    nowId = current_user.get_id()
    if userId == '':
        return jsonify({'result': 0})
    follower = User.get_user_info(userId)['FOLLOWER']

    if nowId in follower: #팔로우 여부 반환
        return jsonify({'result': 1, 'follow': 1})
    else:
        return jsonify({'result': 1, 'follow': 0})

#포스트의 좋아요, 댓글 수 반환
@app.route('/postinfo', methods = ['get', 'post'])
def postinfo():
    index = request.args.get('index', 1)
    with con.cursor() as cur:
        data = cur.execute(
            "SELECT MB_EMAIL FROM LIKE_POSTS WHERE POST_IDX=:1",
            (index,)
        ).fetchall()
        l = ([d[0] for d in data] if data != None else [])
        data = cur.execute(
            "SELECT CMT_IDX FROM COMMENTS WHERE POST_IDX=:1 ORDER BY COMMENTED_AT",
            (index,)
        ).fetchall()
        c = ([d[0] for d in data] if data != None else [])
    return jsonify({'like': l, 'cmt': c})

#새 포스트 개설 동작
@app.route('/newpost', methods = ['get', 'post'])
@login_required
def newPost():
    params = request.get_json()
    content = params['CONTENT']
    imgId = params['IMG_ID']
    index = params['INDEX']
    
    #유효성 검사
    if content == '':
        return jsonify({"result": 0})

    #포스트 내용 임베딩
    embedding = HFmodel.embed_query(content)

    #포스트 내용 db에 저장
    with con.cursor() as cur:
        cur.execute(
            "INSERT INTO POSTS VALUES(0, :1, :2, SYSDATE, 0, :3, :4, :5)",
            ("TITLE", content, current_user.get_id(), index, imgId)
        )
        postId = cur.execute("SELECT MAX(POST_IDX) FROM POSTS").fetchone()[0]
    con.commit()

    #임베딩한 벡터 저장, 사용자, 챌린지 벡터에 반영
    pcon = dbcp.getconn()
    with pcon.cursor() as pcur:
        pcur.execute(
            f"INSERT INTO POSTEMBEDDINGS VALUES('{postId}', '{embedding}'::vector)"
        )
        pcur.execute(
            f"update uservector set embedding = uservector.embedding+'{embedding}'::vector where id='{current_user.get_id()}'"
        )
        pcur.execute(
            f"update challengeembeddings set embedding = challengeembeddings.embedding+'{embedding}'::vector where id={index}"
        )
    pcon.commit()
    dbcp.putconn(pcon)
    return jsonify({"result": 1})

#포스트 좋아요 동작
@app.route('/likepost', methods=['get', 'post'])
@login_required
def likePost():
    postIdx = request.args.get('id', -1)
    #유효성 검사
    if postIdx == -1:
        return jsonify({"result": 0})

    with con.cursor() as cur: #좋아요 중인지 확인
        res = cur.execute(
            "select post_idx from like_posts where mb_email=:1 and post_idx=:2",
            (current_user.get_id(), postIdx)
        ).fetchone()

        if res is None: #좋아요 안된 상태면
            cur.execute(
                "insert into like_posts values (:1, :2)",
                    (postIdx, current_user.get_id())
                ) #좋아요 추가
        else:
            cur.execute(
                "delete from like_posts where mb_email=:1 and post_idx=:2",
                (current_user.get_id(), postIdx)
            ) #좋아요 해제
        likenum = cur.execute(
            "select count(post_idx) from like_posts where post_idx=:1",
            (postIdx,)
            ).fetchone()[0]
    con.commit()
    return jsonify({"result": 1, "likenum": likenum}) #좋아요 수 반환

#댓글 좋아요 동작
@app.route('/likecmt', methods=['get', 'post'])
@login_required
def likeCmt():
    cmtIdx = request.args.get('id', -1)
    #유효성 검사
    if cmtIdx == -1:
        return jsonify({"result": 0})
        
        with con.cursor() as cur:
            res = cur.execute(
                "select cmt_idx from like_comments where mb_email=:1 and cmt_idx=:2",
            (   current_user.get_id(), cmtIdx)
            ).fetchone()
        
        if res is None: #좋아요 안된 상태면
            cur.execute(
                "insert into like_comments values (:1, :2)",
                (current_user.get_id(), cmtIdx)
            ) #좋아요 추가
        else:
            cur.execute(
                "delete from like_comments where mb_email=:1 and cmt_idx=:2",
                (current_user.get_id(), cmtIdx)
            ) #좋아요 해제
        likenum = cur.execute(
                "select count(cmt_idx) from like_comments where cmt_idx=:1",
                (cmtIdx,)
            ).fetchone()[0]
    con.commit()
    return jsonify({"result": 1, "likenum": likenum}) #좋아요 수 반환
       
#포스트 삭제 동작
@app.route('/deletepost', methods = ['get', 'post'])
@login_required
def delPost():
    index = request.args.get('index', -1)
    postInfo = Post.get_post_info(index)
    #유효성 검사
    if postInfo == None:
        return jsonify({"result": 0})
    if postInfo['WRITER'] != current_user.get_id():
        return jsonify({"result": 0})

    #포스트에 달린 댓글의 좋아요, 댓글, 포스트의 좋아요, 포스트 순으로 삭제(foreign key 문제)
    with con.cursor() as cur:
        cur.execute(
            "delete from like_comments where cmt_idx in (select cmt_idx from comments where post_idx=:1)",
            (index,)
        )
        cur.execute(
            "delete from comments where post_idx=:1",
            (index,)
        )   
        cur.execute(
            "delete from like_posts where post_idx=:1",
            (index,)
        )
        cur.execute(
            "delete from posts where post_idx=:1",
            (index,)
        )
    con.commit()
    
    #포스트의 임베딩값 삭제
    pcon = dbcp.getconn()
    with pcon.cursor() as pcur:
        pcur.execute(
            f"delete from postembeddings where id={index}"
        )
    pcon.commit()
    dbcp.putconn(pcon)
    return jsonify({"result": 1})

#댓글 수정 동작
@app.route('/updatecomment', methods = ['get', 'post'])
@login_required
def updatecmt():
    index = request.args.get('index', -1)
    content = request.args.get('content', '-')
    cmtInfo = Comment.get_comment_info(index)
    #유효성 확인
    if cmtInfo == None or content == None:
        return jsonify({result: 0})
    if cmtInfo['WRITER'] != current_user.get_id():
        return jsonify({result: 0})
    #db 업데이트
    with con.cursor() as cur:
        cur.execute(
            "update comments set cmt_content=:1 where cmt_idx=:2",
            (content, index)
        )
    con.commit()
    return jsonify({result: 1})

#댓글 삭제 동작
@app.route('/deletecomment', methods = ['get', 'post'])
@login_required
def delcmt():
    index = request.args.get('index', -1)
    cmtInfo = Comment.get_comment_info(index)
    #유효성 확인
    if cmtInfo == None:
        return jsonify({"result": 0})
    if cmtInfo['WRITER'] != current_user.get_id():
        return jsonify({"result": 0})
    #댓글 좋아요, 댓글 순으로 삭제 (foreign key 문제)
    with con.cursor() as cur:
        cur.execute(
            "delete from like_comments where cmt_idx=:1",
            (index,)
        )
        cur.execute(
            "delete from comments where cmt_idx=:1",
            (index,)
        )
    con.commit()
    return jsonify({"result": 1})

#댓글의 정보 반환
@app.route('/commentinfo', methods = ['get', 'post'])
def commentinfo():
    index = request.args.get('index', 1)
    return jsonify(Comment.get_comment_info(index))

#댓글 작성 동작
@app.route('/newcomment', methods = ['get', 'post'])
@login_required
def newComment():
    params = request.get_json()
    content = params['CONTENT']
    index = params['INDEX']
    #유효성 확인
    if content == '':
        return jsonify({"result": 0})
    #db에 반영
    with con.cursor() as cur:
        cur.execute(
            "INSERT INTO COMMENTS VALUES(0, :1, :2, SYSDATE, :3)",
            (index, content, current_user.get_id())
        )
        con.commit()
    return jsonify({"result": 1})

#포스트 로딩(메인페이지)
@app.route('/nextpost', methods = ['get', 'post'])
@login_required
def nextfeed():
    userId = current_user.get_id()
    #유효성 확인
    mn = int(request.args.get('feednum', -1))+1
    if mn == 0:
        return jsonify({'result': 0})
    #로딩해야할 포스트 범위
    mx = mn +19
    #포스트 범위에 맞게 DB에서 목록 호출
    with con.cursor() as cur:
        data = cur.execute(
            f"select * from (SELECT row_number() over (order by posted_at DESC) num, POSTS.* FROM POSTS WHERE MB_EMAIL IN (SELECT FOLLOW FROM FOLLOWS WHERE FOLLOWER=:1) OR MB_EMAIL=:1 ORDER BY POSTED_AT DESC) where num between {mn} and {mx}",
            (userId,)
        ).fetchall()
    postInfo = {str(d[1]): postdata2dict(d[1:]) for d in data}
    userInfo = {postInfo[i]['WRITER']: User.get_user_info_simple(postInfo[i]['WRITER']) for i in postInfo}
    key = [d[1] for d in data]
    return jsonify({
        'key':key, #포스트 순서
        'postInfo': postInfo, #목록 범위의 포스트 정보
        'userInfo': userInfo #포스트 작성자에 대한 정보
    })

#포스트 로딩(챌린지페이지)
@app.route('/chalnextpost', methods = ['get', 'post'])
@login_required
def chalnextfeed():
    index = request.args.get('index', -1)
    mn = int(request.args.get('feednum', -1))+1
    #유효성 확인
    if mn == 0 or index == -1:
        return jsonify({'result': 0})
    #로딩해야할 포스트 범위
    mx = mn +19
    #포스트 범위에 맞게 DB에서 목록 호출
    with con.cursor() as cur:
        data = cur.execute(
            f"select * from (SELECT row_number() over (order by posted_at DESC) num, POSTS.* FROM POSTS WHERE CHAL_IDX=:1 ORDER BY POSTED_AT DESC) where num between {mn} and {mx}",
            (index,)
        ).fetchall()
    postInfo = {str(d[1]): postdata2dict(d[1:]) for d in data}
    userInfo = {postInfo[i]['WRITER']: User.get_user_info_simple(postInfo[i]['WRITER']) for i in postInfo}
    key = [d[1] for d in data]
    return jsonify({
        'key':key, #포스트 순서
        'postInfo': postInfo, #목록 범위의 포스트 정보
        'userInfo': userInfo #포스트 작성자에 대한 정보
    })

#포스트 로딩(유저페이지)
@app.route('/usernextpost', methods = ['get', 'post'])
def usernextfeed():
    userId = request.args.get('user', '')
    mn = int(request.args.get('feednum', -1))+1
    #유효성 검사
    if mn == 0 or userId == '':
        return jsonify({'result': 0})
    #로딩할 포스트 범위
    mx = mn +19
    #포스트 범위에 맞게 DB에서 목록 호출
    with con.cursor() as cur:
        data = cur.execute(
            f"select * from (SELECT row_number() over (order by posted_at desc) num, POSTS.* FROM POSTS WHERE MB_EMAIL=:1 ORDER BY POSTED_AT DESC) where num between {mn} and {mx}",
            (userId,)
        ).fetchall()
    postInfo = {str(d[1]): postdata2dict(d[1:]) for d in data}
    userInfo = {userId: User.get_user_info_simple(userId)}
    key = [d[1] for d in data]
    return jsonify({
        'key':key, #포스트 순서
        'postInfo': postInfo, #목록 범위의 포스트 정보
        'userInfo': userInfo #포스트 작성자에 대한 정보
    })

#댓글 로딩
@app.route('/nextcmt', methods = ['get', 'post'])
def nextcmt():
    index = request.args.get('post','')
    mn = int(request.args.get('cmtnum', -1))
    #유효성 검사
    if mn == -1 or index == '':
        return jsonify({'result': 0})
    #로딩할 포스트 범위
    mx = mn +9
    #포스트 범위에 맞게 DB에서 목록 호출
    with con.cursor() as cur:
        data = cur.execute(
            f"select * from (select row_number() over (order by commented_at desc) num, comments.* from comments where post_idx=:1 order by commented_at desc) where num between {mn} and {mx}",
            (index,)
        ).fetchall()
    cmtInfo = {d[1]: cmtdata2dict(d[1:]) for d in data}
    userList = [cmtInfo[i]['WRITER'] for i in cmtInfo]
    userInfo = {i: User.get_user_info_simple(i) for i in userList}
    return jsonify({
        'cmtInfo': cmtInfo, #댓글 정보
        'userInfo': userInfo, #댓글 작성자 정보
        'key': [i for i in cmtInfo], #댓글 순서
        'cuser': current_user.get_id() if current_user.is_authenticated else '' #조회자 정보(front 동작에 필요)
    })

#챌린지 참여 기록
@app.route('/challengerecord', methods = ['get', 'post'])
def challengerecord():
    params = request.get_json()
    index = params['INDEX']
    userId = params['USER_ID']
    #참여일부터 오늘까지의 참여 여부 호출
    today = datetime.datetime.now()
    with con.cursor() as cur:
        joinedAt = cur.execute(
            'select joined_at from join_challenges where mb_email=:1 and chal_idx=:2',
            (userId, index)
        ).fetchone()[0]
        record = cur.execute(
            'select posted_at from posts where mb_email=:1 and chal_idx=:2 and posted_at between :3 and :4',
            (userId, index, joinedAt, today)
        ).fetchall()
    record = set([r[0].strftime('%Y-%m-%d') for r in record])
    return jsonify({
        'record': list(record), #참여 기록
        "joined_at": joinedAt.strftime('%Y-%m-%d') #참여일
    })

#종료한 챌린지 참여 기록
@app.route('/endchallengerecord', methods = ['get', 'post'])
def endchallengerecord():
    params = request.get_json()
    index = params['INDEX']
    userId = params['USER_ID']
    #참여일부터 종료일까지의 참여여부 호출
    with con.cursor() as cur:
        dateDue = cur.execute(
            'select joined_at, ended_at, end_idx from end_challenges where mb_email=:1 and chal_idx=:2',
            (userId, index)
        ).fetchall()
        records = [list(set([rec[0].strftime('%Y-%m-%d') for rec in cur.execute(
            'select posted_at from posts where mb_email=:1 and chal_idx=:2 and posted_at between :3 and :4',
            (userId, index, due[0], due[1])
        ).fetchall()])) for due in dateDue]
        
    return jsonify({
        'record': records, #참여 기록
        'due': [[date.strftime('%Y-%m-%d') for date in due[:2]] for due in dateDue], #참여일, 종료일
        'end_idx': [idx[2] for idx in dateDue] #챌린지 인덱스
    })

#검색 동작
@app.route('/search/<key>', methods = ['get', 'post'])
def searchKeyword(key):
    keyword = request.args.get('q', '')
    if keyword == '':
        return ''
    if key not in ('challenges', 'posts', 'members'):
        return ''
    #key따라 검색 쿼리 설정
    idx = {
        'challenges': 'chal_idx',
        'posts': 'post_idx',
        'members': 'mb_email'
    }
    col = {
        'challenges': 'chal_title',
        'posts': 'post_content',
        'members': 'mb_nick'
    }
    embTable = {
        'challenges': 'challengeembeddings',
        'posts': 'postembeddings',
        'members': 'uservector'
    }
    #keyword가 포함된 내용 호출
    with con.cursor() as cur:
        searchList = cur.execute(
            f"select {idx[key]} from {key} where {col[key]} like :1",
            (f'%{keyword}%',)
        ).fetchall()
    searchList = [s[0] for s in searchList]
    #챌린지의 경우 제목 + 본문으로 검색
    if key == 'challenges':
        with con.cursor() as cur:
            searchListMore = cur.execute(
                "select chal_idx from challenges where chal_info like :1",
                (f'%{keyword}%',)
            ).fetchall()
        searchListMore = [s[0] for s in searchListMore if s[0] not in searchList]
        searchList += searchListMore

    #keyword가 포함되지않아도 keyword와 코사인 유사도가 높은 순 출력
    embedding = HFmodel.embed_query(keyword)
    pcon = dbcp.getconn()
    with pcon.cursor() as pcur:
        pcur.execute(
            f"select id from {embTable[key]} order by embedding<=>'{embedding}'::vector limit 10"
        )
        res = pcur.fetchall()
    dbcp.putconn(pcon)
    res = [r[0] for r in res if r[0] not in searchList]
    keys = searchList +res
    if key == 'posts':
        postInfo = {i: Post.get_post_info(i) for i in keys}
        userInfo = {postInfo[i]['WRITER']: User.get_user_info_simple(postInfo[i]['WRITER']) for i in postInfo}
        return jsonify({
            'key': keys,
            'postInfo': postInfo,
            'userInfo': userInfo
        })
    if key == 'challenges':
        chalInfo = {i: Challenge.get_challenge_info(i) for i in keys}
        return jsonify({
            'key': keys,
            'chalInfo': chalInfo
        })
    if key == 'members':
        userInfo = {i: User.get_user_info_simple(i) for i in keys}
        return jsonify({
            'key': keys,
            'userInfo': userInfo,
            'isLogin': current_user.is_authenticated,
            'follow': current_user.get_follow() if current_user.is_authenticated else [],
            'follower': current_user.get_follower() if current_user.is_authenticated else [],
            'cuser': current_user.get_id() if current_user.is_authenticated else []
        })
        
#추천 챌린지 로딩
@app.route('/nextrecchal', methods = ['get', 'post'])
def nextrecchal():
    mn = int(request.args.get('chalnum', -1))
    if mn == -1:
        return jsonify({'result': 0})
    if current_user.is_authenticated:
        userId = current_user.get_id()
        pcon = dbcp.getconn()
        with pcon.cursor() as pcur:
            pcur.execute(
                f"select id, embedding<=>(select embedding from uservector where id='{userId}') from challengeembeddings"
            )
            res = pcur.fetchall()
        dbcp.putconn(pcon)
        score = {r[0]: 1-r[1] for r in res}
    else:
        with con.cursor() as cur:
            res = cur.execute(
                "select chal_idx from challenges"
            ).fetchall()
        score = {r[0]: 0 for r in res}
    
    if current_user.is_authenticated:
        for i in current_user.get_joinCh():
            score[i] = 0
    score = sorted(score, key=lambda x: -score[x])
    chalList = score[mn:mn+4]
    return jsonify({
        'chalInfo': {i: Challenge.get_challenge_info(i) for i in chalList},
        'key': chalList
    })

#좋아요 챌린지 로딩
@app.route('/nextlikechal', methods = ['get', 'post'])
@login_required
def nextlikechal():
    mn = int(request.args.get('chalnum', -1))
    if mn == -1:
        return jsonify({'result': 0})
    with con.cursor() as cur:
        res = cur.execute(
            "select * from challenges where chal_idx in (select chal_idx from like_challenges where mb_email=:1)",
            (current_user.get_id(),)
        ).fetchall()
    res = res[mn:mn+4]
    return jsonify({
        'chalInfo': {i[0]: chaldata2dict(i, auto=True) for i in res},
        'key': [i[0] for i in res]
    })
    
# endregion

c:\Users\SMHRD\anaconda3\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


: 

In [4]:
# oracledb.init_oracle_client(lib_dir='C:\oracledb\instantclient_19_23')
oracledb.init_oracle_client()
dbcp = psycopg2.pool.SimpleConnectionPool(1, 20,
    host="aws-0-ap-northeast-2.pooler.supabase.com",
    dbname="postgres",
    user="postgres.zdtlvavtqrtrixbfxurz",
    password="dvRij1Vb7NU8Wn1f",
    port=6543)
with oracledb.connect(
    user="campus_24SW_LI_p2_1",
    password="smhrd1",
    dsn="project-db-cgi.smhrd.com:1524/xe") as con:
    app.run(host=host, port=5050)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://192.168.219.51:5050
Press CTRL+C to quit
192.168.219.51 - - [10/Jul/2024 15:37:17] "GET /likepost?id=502 HTTP/1.1" 200 -
192.168.219.51 - - [10/Jul/2024 15:37:18] "GET /likepost?id=502 HTTP/1.1" 200 -
192.168.219.51 - - [10/Jul/2024 15:38:47] "POST /newcomment HTTP/1.1" 200 -
192.168.219.51 - - [10/Jul/2024 15:38:47] "GET /nextcmt?post=502&cmtnum=0 HTTP/1.1" 200 -
192.168.219.51 - - [10/Jul/2024 15:38:48] "GET /deletecomment?index=954 HTTP/1.1" 200 -
192.168.219.51 - - [10/Jul/2024 15:38:49] "GET /nextcmt?post=502&cmtnum=0 HTTP/1.1" 200 -
192.168.219.51 - - [10/Jul/2024 15:38:50] "GET /challenge?index=66 HTTP/1.1" 200 -
192.168.219.51 - - [10/Jul/2024 15:38:50] "GET /static/css/style.css HTTP/1.1" 304 -
192.168.219.51 - - [10/Jul/2024 15:38:50] "GET /static/js/challenge.js HTTP/1.1" 304 -
192.168.219.51 - - [10/Jul/2024 15:38:50] "GET /static/img/icons8-user-50.png HTTP/1.1" 304 -
192.168.219.51 - - [10/Jul/2024 15:38:50] "GET /static/img/icons8-trophy-48.png HTTP/